In [1]:
from pathlib import Path
import json
from fishsense_api_workflow_worker.database import Database
from fishsense_api_workflow_worker.config import PG_CONNECTION_STRING
from tqdm import tqdm

/home/chris/Repos/school/e4e/fishsense/fishsense-api-workflow-worker/.venv/lib/python3.13/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "HeadTailLabel" shadows an attribute in parent "SQLModel"
  warnings.warn(
/home/chris/Repos/school/e4e/fishsense/fishsense-api-workflow-worker/.venv/lib/python3.13/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "LaserLabel" shadows an attribute in parent "SQLModel"
  warnings.warn(


In [2]:
PROJECT_10_JSON_PATH = Path("./project-10-at-2025-04-14-20-37-8974c4dc.json")

In [3]:
with open(PROJECT_10_JSON_PATH) as f:
    project_10_data = json.load(f)

project_10_data

[{'id': 33564,
  'annotations': [{'id': 58,
    'completed_by': 4,
    'result': [{'id': 'E6WkooRAi_',
      'type': 'keypointlabels',
      'value': {'x': 47.58695457488871,
       'y': 39.99424912201776,
       'width': 0.22547197916327916,
       'keypointlabels': ['Red Laser']},
      'origin': 'manual',
      'to_name': 'img-1',
      'from_name': 'kp-1',
      'image_rotation': 0,
      'original_width': 3987,
      'original_height': 3016}],
    'was_cancelled': False,
    'ground_truth': False,
    'created_at': '2024-10-24T15:12:58.018502Z',
    'updated_at': '2024-10-24T15:12:58.018514Z',
    'draft_created_at': '2024-10-24T15:12:57.018123Z',
    'lead_time': 15.71,
    'prediction': {},
    'result_count': 0,
    'unique_id': 'b0d60270-e8a6-4519-889d-b5782a0ecdeb',
    'import_id': None,
    'last_action': None,
    'task': 33564,
    'project': 10,
    'updated_by': 4,
    'parent_prediction': None,
    'parent_annotation': None,
    'last_created_by': None}],
  'drafts': [

In [4]:
database = Database(PG_CONNECTION_STRING)

In [ ]:
laser_labels = await database.select_laser_labels()

for laser_label in tqdm(laser_labels):
    label_studio_label = [t for t in project_10_data if t["id"] == laser_label.label_studio_task_id]

    if not label_studio_label:
        continue

    label_studio_label = label_studio_label[0]

    user_id: int | None = None
    if label_studio_label["annotations"]:
        user_label_studio_id = label_studio_label["annotations"][0]["completed_by"]
        user = await database.select_user_by_label_studio_id(user_label_studio_id)
        user_id = user.id if user else None

    laser_label.user_id = user_id
    laser_label.json = label_studio_label

    if label_studio_label["annotations"] and label_studio_label["annotations"][0]["result"]:
        laser_label.label = label_studio_label["annotations"][0]["result"][0]["value"]["keypointlabels"][0]

    await database.insert_or_update_laser_label(laser_label)